   # TP53: Effects of Mutation

<b>Standard imports for playing with and plotting data frames.</b>

In [2]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

<b>Import CPTAC data</b>

In [3]:
import CPTAC

Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Data...

 ******PLEASE READ******


In [4]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

### List of proteins that interact with TP53 (according to Uniprot)

In [13]:
protList = ['AXIN1', 'EP300', 'HRMT1L2', 'CARM1', 'TAF1', 'ING4', 'CABLES1', 
            'TP73', 'HIPK1', 'HIPK2', 'TP53INP1', 'WWOX', 'HCV', 'USP7', 'SYVN1', 'HSP90AB1',
           'CHD8', 'ARMC10', 'BANP', 'CDKN2AIP', 'NUAK1', 'STK11', 'LKB1', 'UHRF2', 'E4F1', 'YWHAZ',
           'MAML1', 'MKRN1', 'PML', 'MDM2', 'FBXO42', 'ATM', 'PP2R2A', 'AURKA', 'DAXX', 'BRD7', 'TRIM24',
           'L3MBTL1', 'GRK5', 'CAK', 'PTK2B', 'PYK2', 'MDM2', 'PRKCG', 'PPIF', 'KAT6A', 'UBC9', 'ZNF385B',
           'ZNF385A', 'ANKRD2', 'RFFL', 'RNF34', 'MTA1', 'COP1', 'CCAR2', 'MORC3', 'POU4F2', 'AFG1L', 'UBD',
           'TAF6', 'HPV', 'UL123', 'E1B-55K']

### Proteome abundance of proteins that interact with TP53

In [14]:
gene = 'TP53'
sigList = [];

pcutoff = 0.05/len(protList)

genedf = somatic_mutations[gene].to_frame()
for protein in protList:
    if protein in proteomics.columns:
        proteindf = proteomics[protein].to_frame()
        proteindfName = protein + " protein"
        proteindf.columns = [proteindfName]
        cross = genedf.add(proteindf, fill_value=0).dropna(axis=0)
        mutated = cross.loc[cross[gene] == 1.0]
        wt = cross.loc[cross[gene] == 0.0]
        ttest = scipy.stats.ttest_ind(mutated[proteindfName], wt[proteindfName])
        if ttest[1] <= pcutoff:
            sigList.append(protein)
            print("Test for " + protein + ": ")
            print(ttest)

Test for CABLES1: 
Ttest_indResult(statistic=-4.533885225055603, pvalue=1.6398425638878636e-05)
Test for HSP90AB1: 
Ttest_indResult(statistic=3.775761395509814, pvalue=0.000273573248009503)
Test for CHD8: 
Ttest_indResult(statistic=4.65599574118626, pvalue=1.0132380205138297e-05)
Test for AURKA: 
Ttest_indResult(statistic=4.2121494006965365, pvalue=8.335995102753076e-05)
Test for DAXX: 
Ttest_indResult(statistic=4.139121723641875, pvalue=7.383403320014249e-05)


### List of significantly affected proteins

In [15]:
print(sigList)

['CABLES1', 'HSP90AB1', 'CHD8', 'AURKA', 'DAXX']


### Phosphoproteome abundance of interacting proteins

In [26]:
gene = 'TP53'
genedf = somatic_mutations[gene].to_frame()
sigPhosResults = [];

for protein in protList:
    sites = phos.filter(regex=protein)
    genedf = genedf.add(sites, fill_value=0)

mutated = genedf.loc[genedf[gene] == 1.0]
wt = genedf.loc[genedf[gene] == 0.0]

pcutoff = 0.05 / len(genedf.columns)

for loc in genedf.columns:
    if not loc == gene:
        mutsitedf = mutated[[gene, loc]].dropna()
        wtsitedf = wt[[gene, loc]].dropna()
        ttest = scipy.stats.ttest_ind(mutsitedf[loc], wtsitedf[loc])
        if(ttest[1] <= pcutoff):
            sigPhosResults.append(loc)
            print('Results for ' + loc + ': ')
            print(ttest)
 

Results for HSP90AB1-S226: 
Ttest_indResult(statistic=4.83818141889424, pvalue=4.875355910124854e-06)
Results for HSP90AB1-S255: 
Ttest_indResult(statistic=5.523615626841696, pvalue=2.7326572771063515e-07)
Results for HSP90AB1-S261: 
Ttest_indResult(statistic=5.578991724475906, pvalue=2.147645387372064e-07)


### List of significantly affected phosphorylation sites

In [27]:
print(sigPhosResults)

['HSP90AB1-S226', 'HSP90AB1-S255', 'HSP90AB1-S261']
